# Libs and pre-definitions

### Bibliotecas padrões python e utils pytorch

In [1]:
import torch
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Lambda, Compose, Normalize
from collections import defaultdict
from torch.utils.data import random_split
import copy

In [2]:
# Define o computador utilizado como cuda (gpu) se existir ou cpu caso contrário
print(torch.cuda.is_available())
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

True


### Bibliotecas desenvolvidas

https://github.com/lfpc/Uncertainty_Estimation

In [3]:
from NN_utils import *
from NN_utils.train_and_eval import *
from uncertainty import train_NN_with_g
from uncertainty.losses import penalized_uncertainty
from NN_models import Model_CNN
import uncertainty.comparison as unc_comp
import uncertainty.quantifications as unc

## Data download and transforms

In [27]:
transforms_train = transforms.Compose([
                    transforms.ToTensor(),
                    #transforms.RandomCrop(32, padding=4),
                    #transforms.RandomHorizontalFlip(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
transforms_test = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),])

In [28]:
training_data = datasets.CIFAR10(
root="data",
 train=True,
 download=True,
transform=transforms_train)

test_data = datasets.CIFAR10(
root="data",
train=False,
download=True,
transform=transforms_test)

train_size = int(0.85*len(training_data))
val_size = len(training_data) - train_size
training_data, validation_data = random_split(training_data, [train_size, val_size])

#validation_data = copy.deepcopy(validation_data)
#validation_data.dataset.transforms = transforms_test

Files already downloaded and verified
Files already downloaded and verified


In [29]:
batch_size = 12
train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle = True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size,shuffle = False)
test_dataloader = DataLoader(test_data, batch_size=100)

## NN classes and Trainer class

### Definição da classe da rede neural

In [7]:
# Define model
class Model_CNN(nn.Module):
    """CNN."""

    def __init__(self,n_classes=10):
        """CNN Builder."""
        super().__init__()
        
        conv_layer = [

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=int(16), kernel_size=3, padding=1),
            nn.BatchNorm2d(int(16)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=int(16), out_channels=int(32), kernel_size=3, padding=1),
            nn.Dropout(p=0.2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

        ]
        
        fc_layer = [
            nn.Flatten(),
            nn.Dropout(p=0.2),
            nn.Linear(8192, int(1024)),
            nn.ReLU(inplace=True),
            nn.Linear(int(1024), int(512)),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2)]
        
        main_layer = conv_layer+fc_layer
        
        self.main_layer = nn.Sequential(*main_layer)
        
        self.classifier_layer = nn.Sequential(
            nn.Linear(int(512), n_classes),
            nn.LogSoftmax(dim=1)
        )


    def forward(self, x):
        """Perform forward."""
    
        x = self.main_layer(x)


        y = self.classifier_layer(x)

        y = y.float()

        
        return y


In [8]:
# Define model
class Model_CNN_with_g(nn.Module):
    """CNN."""

    def __init__(self,n_classes=10):
        """CNN Builder."""
        super().__init__()
        
        self.return_g = False
        
        conv_layer = [

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=int(16), kernel_size=3, padding=1),
            nn.BatchNorm2d(int(16)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=int(16), out_channels=int(32), kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

        ]
        
        fc_layer = [
            nn.Flatten(),
            nn.Dropout(p=0.2),
            nn.Linear(8192, int(1024)),
            nn.ReLU(inplace=True),
            nn.Linear(int(1024), int(512)),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2)]
        
        
        main_layer = conv_layer+fc_layer
        
        self.main_layer = nn.Sequential(*main_layer)
        

        self.classifier_layer = nn.Sequential(
            nn.Linear(int(512), n_classes),
            nn.LogSoftmax(dim=1)
        )
        
        '''self.fc_g_layer = nn.Sequential(
            
            nn.Linear(int(512), 1),
            nn.Sigmoid()
        )'''
        
        self.g_layer = nn.Sequential(
            
            nn.Linear(int(512), n_classes),
            nn.Softmax(dim=1)
        )


    def forward(self, x):
        """Perform forward."""
    
        x = self.main_layer(x)


        y = self.classifier_layer(x)

        self.g = self.g_layer(x)
        self.g = torch.max(self.g,dim=1).values

        self.g = (self.g).float()
        y = y.float()

        
        if self.return_g:
            return y,self.g
        else:
            return y
    
    def get_g(self):
        return self.g

In [9]:
class Model_CNN_with_g_2(nn.Module):
    """CNN."""

    def __init__(self,n_classes=10):
        """CNN Builder."""
        super().__init__()
        
        self.return_g = False
        
        conv_layer = [

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=int(16), kernel_size=3, padding=1),
            nn.BatchNorm2d(int(16)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=int(16), out_channels=int(32), kernel_size=3, padding=1),
            nn.Dropout(p=0.2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

        ]
        
        fc_layer = [
            nn.Flatten(),
            nn.Dropout(p=0.2),
            nn.Linear(8192, int(1024)),
            nn.ReLU(inplace=True),
            nn.Linear(int(1024), int(512)),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2)]
        
        
        main_layer = conv_layer+fc_layer
        
        self.main_layer = nn.Sequential(*main_layer)
        

        self.classifier_layer = nn.Sequential(
            nn.Linear(int(512), n_classes),
            nn.LogSoftmax(dim=1)
        )
        
        '''self.fc_g_layer = nn.Sequential(
            
            nn.Linear(int(512), 1),
            nn.Sigmoid()
        )'''
        
        self.g_layer = nn.Sequential(
            
            nn.Linear(n_classes, 64),
            nn.ReLU(inplace=True), #tanh
            nn.Dropout(p=0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        """Perform forward."""
    
        x = self.main_layer(x)


        y = self.classifier_layer(x)

        self.g = self.g_layer(y)

        self.g = (self.g).float()
        y = y.float()

        
        if self.return_g:
            return y,self.g
        else:
            return y
    
    def get_g(self):
        return self.g

In [10]:
class Model_CNN_with_g_3(nn.Module):
    """CNN."""

    def __init__(self,n_classes=10):
        """CNN Builder."""
        super().__init__()
        
        self.return_g = False
        
        conv_layer = [

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=int(16), kernel_size=3, padding=1),
            nn.BatchNorm2d(int(16)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=int(16), out_channels=int(32), kernel_size=3, padding=1),
            nn.Dropout(p=0.2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

        ]
        
        fc_layer = [
            nn.Flatten(),
            nn.Dropout(p=0.2),
            nn.Linear(8192, int(1024)),
            nn.ReLU(inplace=True),
            nn.Linear(int(1024), int(512)),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2)]
        
        
        main_layer = conv_layer+fc_layer
        
        self.main_layer = nn.Sequential(*main_layer)
        

        self.classifier_layer = nn.Sequential(
            nn.Linear(int(512), n_classes),
            nn.LogSoftmax(dim=1)
        )
        
        '''self.fc_g_layer = nn.Sequential(
            
            nn.Linear(int(512), 1),
            nn.Sigmoid()
        )'''
        
        self.g_layer = nn.Sequential(
            
            nn.Linear(512, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        """Perform forward."""
    
        x = self.main_layer(x)


        y = self.classifier_layer(x)

        self.g = self.g_layer(x)

        self.g = (self.g).float()
        y = y.float()

        
        if self.return_g:
            return y,self.g
        else:
            return y
    
    def get_g(self):
        return self.g

In [11]:
class Model_CNN_with_g_4(nn.Module): #realizar concatenação de x com y e etc
    """CNN."""

    def __init__(self,n_classes=10):
        """CNN Builder."""
        super().__init__()
        
        self.return_g = False
        
        conv_layer = [

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=int(16), kernel_size=3, padding=1),
            nn.BatchNorm2d(int(16)),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=int(16), out_channels=int(32), kernel_size=3, padding=1),
            nn.Dropout(p=0.2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

        ]
        
        fc_layer = [
            nn.Flatten(),
            nn.Dropout(p=0.2),
            nn.Linear(8192, int(1024)),
            nn.ReLU(inplace=True),
            nn.Linear(int(1024), int(512)),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.2)]
        
        
        main_layer = conv_layer+fc_layer
        
        self.main_layer = nn.Sequential(*main_layer)
        

        self.classifier_layer = nn.Sequential(
            nn.Linear(int(512), n_classes),
            nn.LogSoftmax(dim=1)
        )
        
        '''self.fc_g_layer = nn.Sequential(
            
            nn.Linear(int(512), 1),
            nn.Sigmoid()
        )'''
        
        self.g_layer_1 = nn.Sequential(
            
            nn.Linear(512, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 10)
        )
        self.g_layer_2 = nn.Sequential(
            
            nn.Linear(20, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )


    def forward(self, x):
        """Perform forward."""
    
        x = self.main_layer(x)


        y = self.classifier_layer(x)

        self.g = self.g_layer_1(x)
        self.g = torch.cat((self.g,y),dim=1)
        self.g = self.g_layer_2(self.g)

        self.g = (self.g).float()
        y = y.float()
        
        if self.return_g:
            return y,self.g
        else:
            return y
    
    def get_g(self):
        return self.g

### Definição das classes de treinamento

In [12]:
class hist_train():
    def __init__(self,model,loss_criterion,data):
        
        self.model = model
        self.loss_criterion = loss_criterion
        self.data = data
        
        self.acc_list = []
        self.loss_list = []
    
        
    def update_hist(self,data = None):
        
        if data is None:
            data = self.data
        
        with torch.no_grad():
            loss = 0
            hits = 0
            for image,label in data:
                image,label = image.to(dev),label.to(dev)
                output = self.model(image)
                loss += self.loss_criterion(output,label).item()
                hits += correct_total(output,label)
            
            acc = hits/len(data.dataset)
            loss = loss/len(data.dataset)
            self.acc_list.append(acc)
            self.loss_list.append(loss)
                
    def __str__(self):
        pass
            
                
class hist_train_g(hist_train):
    def __init__(self,model,loss_criterion,data,c = 0):
        super().__init__(model,loss_criterion,data)
        
        self.c = c
        self.g_list = []
        if c>0:
            self.acc_c_g = []
            self.acc_c_mcp = []
            
    def update_hist(self,data = None):
        
        if data is None:
            data = self.data
            
        with torch.no_grad():

            label, output, g = accumulate_results(self.model,data)
            loss = self.loss_criterion(output,label).item()
            output = torch.exp(output)
            acc = correct_total(output,label)/label.size(0)

            self.g_list.append(torch.mean(g).item())
            self.acc_list.append(acc)
            self.loss_list.append(loss)

            mcp = unc.MCP_unc(output)

            if self.c>0:
                self.acc_c_g.append(unc_comp.acc_coverage(output,label,1-g,self.c))
                self.acc_c_mcp.append(unc_comp.acc_coverage(output,label,mcp,self.c))
            
        
    
class Trainer():
    def __init__(self,model,optimizer,loss_criterion,training_data,validation_data):
        super().__init__()
        
        loss_crit = copy.deepcopy(loss_criterion.criterion)
        loss_crit.reduction = 'sum'
        
        self.hist_train = hist_train(model,loss_crit,training_data)
        self.hist_val = hist_train(model,loss_crit,validation_data)
        self.model = model
        self.optimizer = optimizer
        self.loss_fn = loss_criterion

    def fit(self,data,n_epochs):
        loss_criterion = copy.deepcopy(self.loss_fn.criterion)
        loss_criterion.reduction = 'mean'
        for epoch in range(1,n_epochs+1):
            loss = train_NN(self.model,self.optimizer,data,loss_criterion,1, print_loss = False, set_train_mode = True)
            print('Epoch ', epoch+1, ', loss = ', loss)
            self.hist_train.update_hist()
            self.hist_val.update_hist()
            
            
            
class Trainer_with_g(Trainer):
    def __init__(self,model,optimizer,loss_fn,training_data,validation_data, c = 0.2):
        super().__init__(model,optimizer,loss_fn,training_data,validation_data)
        
        loss_criterion = copy.deepcopy(loss_fn.criterion)
        loss_criterion.reduction = 'mean'
        
        self.hist_train = hist_train_g(model,loss_criterion,training_data, c=c)
        self.hist_val = hist_train_g(model,loss_criterion,validation_data,c=c)
    
    def fit_all(self,data,n_epochs):
        unfreeze_params(self.model)
        for epoch in range(1,n_epochs+1):
            train_NN_with_g(self.model,self.optimizer,data,self.loss_fn,n_epochs=1, print_loss = True,set_train_mode = True)
            self.hist_train.update_hist()
            self.hist_val.update_hist()
            self.loss_fn.update_L0(self.hist_train.loss_list[-1])

    def fit_g(self,data,n_epochs,ignored_layers = ['main_layer','classifier_layer']):
        
        loss = self.hist_val.loss_list[-1]
        self.loss_fn.update_L0(loss)
        ignore_layers(self.model,ignored_layers, reset = True)
        for epoch in range(1,n_epochs+1):
            train_NN_with_g(self.model,self.optimizer,data,self.loss_fn,n_epochs=1, print_loss = True,set_train_mode = False)
            self.hist_train.update_hist()
            self.hist_val.update_hist()
            ignore_layers(self.model,ignored_layers, reset = False)
            loss =  self.hist_train.loss_list[-1]#self.hist_val.loss_list[-1]
            self.loss_fn.update_L0(loss)


# Testes e treinamentos

### Definição da perda

In [13]:
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(2))

In [14]:
class aux_loss_fs(nn.Module):
    
    def __init__(self,loss_criterion):
        super().__init__()
        self.L0 = 0
        self.criterion = loss_criterion
        
    def forward(self, y_pred,g,y_true):
        g = g.view(-1)
        y_pred = torch.exp(y_pred)
        right = correct_class(y_pred,y_true).float()
        #loss = torch.square(g.view(-1)-MCP)
        loss = self.criterion(g,right)
        loss = torch.mean(loss)
        return loss
    
    def update_L0(self,new_L0):
        with torch.no_grad():
            self.L0 = new_L0

In [15]:
class aux_loss(nn.Module):
    
    def __init__(self,loss_criterion):
        super().__init__()
        self.L0 = 0
        self.criterion = loss_criterion
        
    def forward(self, y_pred,g,y_true):
        g = g.view(-1)
        y_pred = torch.exp(y_pred)
        MCP = unc.get_MCP(y_pred)
        loss = torch.square(g.view(-1)-MCP)
        loss = torch.mean(loss)
        return loss
    
    def update_L0(self,new_L0):
        with torch.no_grad():
            self.L0 = new_L0

### Treinamento dos modelos


#### Classificador

In [ ]:
model = Model_CNN(10).cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(2))

model_trainer = Trainer(model,optimizer,loss_fn, train_dataloader,validation_dataloader)
model_trainer.fit(train_dataloader,70)
state_dict  = model.state_dict()

acc = model_acc(model,train_dataloader)
print('Conjunto de treinamento: acc = ', acc)
acc = model_acc(model,test_dataloader)
print('Conjunto de teste: acc = ', acc)

Epoch  2 , loss =  2.0140752805820767
Epoch  3 , loss =  1.5965657266669864
Epoch  4 , loss =  1.4195810904834716
Epoch  5 , loss =  1.3122157062567406
Epoch  6 , loss =  1.2305067602875273
Epoch  7 , loss =  1.1642795500194603
Epoch  8 , loss =  1.0978246376482126
Epoch  9 , loss =  1.0404561478486847


#### Perda padrão

In [17]:
model_1 = Model_CNN_with_g()
model_1 = model_1.to(dev)
optimizer = torch.optim.SGD(model_1.parameters(), lr=1e-3)

loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_1 = Trainer_with_g(model_1,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
model_trainer_1.fit_all(train_dataloader,200)
acc, g, bce = model_metrics(model_1,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_1,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

Epoch  1 , loss =  2.301543180919649
Epoch  1 , loss =  2.27968191595686
Epoch  1 , loss =  2.2367758080489484
Epoch  1 , loss =  1.9105248117702684
Epoch  1 , loss =  1.6132616058101363
Epoch  1 , loss =  1.4148842599653106
Epoch  1 , loss =  1.32431927582492
Epoch  1 , loss =  1.2696973100380329
Epoch  1 , loss =  1.2121404495062902
Epoch  1 , loss =  1.1628997752486077
Epoch  1 , loss =  1.121644231490265
Epoch  1 , loss =  1.0931199324932024
Epoch  1 , loss =  1.0532309755910123
Epoch  1 , loss =  1.0348145226463634
Epoch  1 , loss =  0.9927366655267341
Epoch  1 , loss =  0.968609496547232
Epoch  1 , loss =  0.9298620669014503
Epoch  1 , loss =  0.9369798226840479
Epoch  1 , loss =  0.8788265892673117
Epoch  1 , loss =  0.8587591381883433
Epoch  1 , loss =  0.827658844635286
Epoch  1 , loss =  0.8010640285029915
Epoch  1 , loss =  0.7776031847107148
Epoch  1 , loss =  0.7575727672411318
Epoch  1 , loss =  0.7355670065554369
Epoch  1 , loss =  0.7174429142335456
Epoch  1 , loss =  0

In [18]:

model_sep = Model_CNN_with_g()
model_sep = model_sep.to(dev)
model_sep.load_state_dict(state_dict,strict = False)
optimizer = torch.optim.SGD(model_sep.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_sep = Trainer_with_g(model_sep,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
model_trainer_sep.hist_train = model_trainer.hist_train
model_trainer_sep.hist_val = model_trainer.hist_val
model_trainer_sep.hist_val.c = 0.2

#model_trainer_sep.fit(train_dataloader,40)
#model_trainer_sep.optimizer = torch.optim.SGD(model_sep.parameters(), lr=1e-2) #testar variações de lr
model_trainer_sep.fit_g(validation_dataloader,200)

acc, g, bce = model_metrics(model_sep,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_sep,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

Epoch  1 , loss =  1.1922246829986571
Epoch  1 , loss =  0.5212113477706909
Epoch  1 , loss =  0.520684111738205
Epoch  1 , loss =  0.5201988306045532
Epoch  1 , loss =  0.5197402727603913
Epoch  1 , loss =  0.5193021864891052
Epoch  1 , loss =  0.5188841783046723
Epoch  1 , loss =  0.5184836725234986
Epoch  1 , loss =  0.5180991847991944
Epoch  1 , loss =  0.5177333681583405
Epoch  1 , loss =  0.5173823194980621
Epoch  1 , loss =  0.5170460584163665
Epoch  1 , loss =  0.5167211390018464
Epoch  1 , loss =  0.5164075161933899
Epoch  1 , loss =  0.5161055882453919
Epoch  1 , loss =  0.5158129669189453
Epoch  1 , loss =  0.5155284389972686
Epoch  1 , loss =  0.5152503711223603
Epoch  1 , loss =  0.5149815976142883
Epoch  1 , loss =  0.5147210410118103
Epoch  1 , loss =  0.514467029094696
Epoch  1 , loss =  0.5142192105770111
Epoch  1 , loss =  0.513979636144638
Epoch  1 , loss =  0.5137450945854187
Epoch  1 , loss =  0.5135151543140412
Epoch  1 , loss =  0.5132906772136688
Epoch  1 , loss

In [19]:
model_2 = Model_CNN_with_g_2()
model_2 = model_2.to(dev)
optimizer = torch.optim.SGD(model_2.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_2 = Trainer_with_g(model_2,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
model_trainer_2.fit_all(train_dataloader,80)

acc, g, bce = model_metrics(model_2,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_2,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

Epoch  1 , loss =  2.127191789671232
Epoch  1 , loss =  1.6735845494748238
Epoch  1 , loss =  1.4710401970743796
Epoch  1 , loss =  1.3494695694617536
Epoch  1 , loss =  1.2341767882876717
Epoch  1 , loss =  1.1163182905641134
Epoch  1 , loss =  1.1269351798454157
Epoch  1 , loss =  1.0452796544128726
Epoch  1 , loss =  1.0293637551809016
Epoch  1 , loss =  1.0198743759845355
Epoch  1 , loss =  1.0063617422983244
Epoch  1 , loss =  0.9414404416020343
Epoch  1 , loss =  0.9235521502696075
Epoch  1 , loss =  0.9403438057638979
Epoch  1 , loss =  0.8773932461597904
Epoch  1 , loss =  0.8708647974108923
Epoch  1 , loss =  0.8496400562973766
Epoch  1 , loss =  0.8099564119020739
Epoch  1 , loss =  0.8564803261143686
Epoch  1 , loss =  0.7804034991386515
Epoch  1 , loss =  0.7698184375639086
Epoch  1 , loss =  0.7552894829088251
Epoch  1 , loss =  0.7433077110990436
Epoch  1 , loss =  0.6959693164457916
Epoch  1 , loss =  0.7153422968374814
Epoch  1 , loss =  0.6715912522820683
Epoch  1 , lo

In [ ]:
#modelo 2 # testar com tanh
model_sep_2 = Model_CNN_with_g_2()
model_sep_2 = model_sep_2.to(dev)
model_sep_2.load_state_dict(state_dict,strict = False)
optimizer = torch.optim.SGD(model_sep_2.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_sep_2 = Trainer_with_g(model_sep_2,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
model_trainer_sep_2.hist_train.acc_list = copy.copy(model_trainer.hist_train.acc_list)
model_trainer_sep_2.hist_train.loss_list = copy.copy(model_trainer.hist_train.loss_list)
model_trainer_sep_2.hist_val.acc_list = copy.copy(model_trainer.hist_val.acc_list)
model_trainer_sep_2.hist_val.loss_list = copy.copy(model_trainer.hist_val.loss_list)
#model_trainer_sep_2.fit(train_dataloader,40)
model_trainer_sep_2.fit_g(train_dataloader,200)

acc, g, bce = model_metrics(model_sep_2,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_sep_2,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

Epoch  1 , loss =  0.2968389642067813
Epoch  1 , loss =  0.29070024498181446
Epoch  1 , loss =  0.27099289209857996
Epoch  1 , loss =  0.2517227002249674
Epoch  1 , loss =  0.2441359755747706
Epoch  1 , loss =  0.23955971370059376
Epoch  1 , loss =  0.23644097687025517
Epoch  1 , loss =  0.23402158949862828
Epoch  1 , loss =  0.23205034803748467
Epoch  1 , loss =  0.23040675888293413
Epoch  1 , loss =  0.22901517063387702
Epoch  1 , loss =  0.2278246636897376
Epoch  1 , loss =  0.22677750012115325
Epoch  1 , loss =  0.22579026433840638
Epoch  1 , loss =  0.22498861181995142
Epoch  1 , loss =  0.2242322897571147
Epoch  1 , loss =  0.22351822954350645
Epoch  1 , loss =  0.22286475645215212
Epoch  1 , loss =  0.22224894637491224
Epoch  1 , loss =  0.2216745160019445
Epoch  1 , loss =  0.22118211413282987
Epoch  1 , loss =  0.2206257650074415
Epoch  1 , loss =  0.2201557231037328
Epoch  1 , loss =  0.21967617805599157
Epoch  1 , loss =  0.21924772803192588
Epoch  1 , loss =  0.218802816105

In [ ]:
model_3 = Model_CNN_with_g_3()
model_3 = model_3.to(dev)
optimizer = torch.optim.SGD(model_3.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_3 = Trainer_with_g(model_3,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
model_trainer_3.fit_all(train_dataloader,80)

acc, g, bce = model_metrics(model_3,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_3,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

In [ ]:
#modelo 3
model_sep_3 = Model_CNN_with_g_3()
model_sep_3 = model_sep_3.to(dev)
model_sep_3.load_state_dict(state_dict,strict = False)
optimizer = torch.optim.SGD(model_sep_3.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_sep_3 = Trainer_with_g(model_sep_3,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
model_trainer_sep_3.hist_train.acc_list = copy.copy(model_trainer.hist_train.acc_list)
model_trainer_sep_3.hist_train.loss_list = copy.copy(model_trainer.hist_train.loss_list)
model_trainer_sep_3.hist_val.acc_list = copy.copy(model_trainer.hist_val.acc_list)
model_trainer_sep_3.hist_val.loss_list = copy.copy(model_trainer.hist_val.loss_list)
#model_trainer_sep_2.fit(train_dataloader,40)
model_trainer_sep_3.fit_g(train_dataloader,200)

acc, g, bce = model_metrics(model_sep_3,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_sep_3,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

In [ ]:
model_4 = Model_CNN_with_g_4()
model_4 = model_4.to(dev)
optimizer = torch.optim.SGD(model_4.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_4 = Trainer_with_g(model_4,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
model_trainer_4.fit_all(train_dataloader,80)

acc, g, bce = model_metrics(model_4,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_4,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

In [ ]:
#modelo 2
model_sep_4 = Model_CNN_with_g_4()
model_sep_4 = model_sep_4.to(dev)
model_sep_4.load_state_dict(state_dict,strict = False)
optimizer = torch.optim.SGD(model_sep_4.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_sep_4 = Trainer_with_g(model_sep_4,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
model_trainer_sep_4.hist_train.acc_list = model_trainer.hist_train.acc_list
model_trainer_sep_4.hist_train.loss_list = model_trainer.hist_train.loss_list
model_trainer_sep_4.hist_val.acc_list = model_trainer.hist_val.acc_list
model_trainer_sep_4.hist_val.loss_list = model_trainer.hist_val.loss_list
#model_trainer_sep_2.fit(train_dataloader,40)
model_trainer_sep_4.fit_g(train_dataloader,200)

acc, g, bce = model_metrics(model_sep_4,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_sep_4,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

#### Perda adaptada

In [ ]:
#modelo 2
model_sep_mcp = Model_CNN_with_g_2()
model_sep_mcp = model_sep_mcp.to(dev)
optimizer = torch.optim.SGD(model_sep_mcp.parameters(), lr=1e-3)
loss_fn = aux_loss(loss_criterion)#penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_sep_mcp = Trainer_with_g(model_sep_mcp,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
model_trainer_sep_mcp.fit(train_dataloader,40)
model_trainer_sep_mcp.fit_g(validation_dataloader,200)

acc, g, bce = model_metrics(model_sep_mcp,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_sep_mcp,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

In [ ]:
#modelo fs
'''model_sep_fs = Model_CNN_with_g_2()
model_sep_fs = model_sep_fs.to(dev)
model_sep_fs.load_state_dict(state_dict,strict = False)
optimizer = torch.optim.SGD(model_sep_fs.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = aux_loss_fs(loss_criterion)#penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_sep_fs = Trainer_with_g(model_sep_fs,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
#model_trainer_sep_fs.fit(train_dataloader,40)
model_trainer_sep_fs.hist_train.acc_list = copy.copy(model_trainer.hist_train.acc_list)
model_trainer_sep_fs.hist_train.loss_list = copy.copy(model_trainer.hist_train.loss_list)
model_trainer_sep_fs.hist_val.acc_list = copy.copy(model_trainer.hist_val.acc_list)
model_trainer_sep_fs.hist_val.loss_list = copy.copy(model_trainer.hist_val.loss_list)'''

model_trainer_sep_fs.loss_fn.criterion = nn.BCELoss()
model_trainer_sep_fs.fit_g(train_dataloader,200)

acc, g, bce = model_metrics(model_sep_fs,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_sep_fs,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

In [ ]:
#modelo fs
model_sep_fs_3 = Model_CNN_with_g_3()
model_sep_fs_3 = model_sep_fs_3.to(dev)
model_sep_fs_3.load_state_dict(state_dict,strict = False)
optimizer = torch.optim.SGD(model_sep_fs_3.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = aux_loss_fs(loss_criterion)#penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_sep_fs_3 = Trainer_with_g(model_sep_fs_3,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
#model_trainer_sep_fs.fit(train_dataloader,40)
model_trainer_sep_fs_3.hist_train.acc_list = copy.copy(model_trainer.hist_train.acc_list)
model_trainer_sep_fs_3.hist_train.loss_list = copy.copy(model_trainer.hist_train.loss_list)
model_trainer_sep_fs_3.hist_val.acc_list = copy.copy(model_trainer.hist_val.acc_list)
model_trainer_sep_fs_3.hist_val.loss_list = copy.copy(model_trainer.hist_val.loss_list)

model_trainer_sep_fs_3.loss_fn.criterion = nn.BCELoss()
model_trainer_sep_fs_3.fit_g(train_dataloader,800)

acc, g, bce = model_metrics(model_sep_fs_3,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_sep_fs_3,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

In [ ]:
#modelo fs
model_sep_fs_4 = Model_CNN_with_g_4()
model_sep_fs_4 = model_sep_fs_4.to(dev)
model_sep_fs_4.load_state_dict(state_dict,strict = False)
optimizer = torch.optim.SGD(model_sep_fs_4.parameters(), lr=1e-3)
loss_criterion = nn.NLLLoss(reduction = 'none')
loss_fn = aux_loss_fs(loss_criterion)#penalized_uncertainty(loss_criterion,np.log(10))

model_trainer_sep_fs_4 = Trainer_with_g(model_sep_fs_4,optimizer,loss_fn, train_dataloader,validation_dataloader,c = 0.2)
#model_trainer_sep_fs.fit(train_dataloader,40)
model_trainer_sep_fs_4.hist_train.acc_list = model_trainer.hist_train.acc_list
model_trainer_sep_fs_4.hist_train.loss_list = model_trainer.hist_train.loss_list
model_trainer_sep_fs_4.hist_val.acc_list = model_trainer.hist_val.acc_list
model_trainer_sep_fs_4.hist_val.loss_list = model_trainer.hist_val.loss_list

model_trainer_sep_fs_4.loss_fn.criterion = nn.BCELoss()
model_trainer_sep_fs_4.fit_g(train_dataloader,800)

acc, g, bce = model_metrics(model_sep_fs,loss_criterion,train_dataloader)
print('Conjunto de treinamento: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')
acc, g, bce = model_metrics(model_sep_fs,loss_criterion,test_dataloader)
print('Conjunto de teste: acc = ', acc, 'média de g = ', g, 'média de bce = ', bce, '\n')

## Plots e análises

In [ ]:
models = {model:model_trainer,
          model_sep_2:model_trainer_sep_2,
          model_sep_4:model_trainer_sep_4,
          model_sep_fs:model_trainer_sep_fs,
          model_sep_fs_3:model_trainer_sep_fs_3,
          model_sep_fs_4:model_trainer_sep_fs_4,
          model_sep_3:model_trainer_sep_3}

In [ ]:
models = {model:model_trainer,
          model_sep:model_trainer_sep,
          model_2:model_trainer_2,
          model_sep_2:model_trainer_sep_2#,
          model_sep_22:model_trainer_sep_22,
          model_sep_mcp:model_trainer_sep_mcp,
          model_sep_fs:model_trainer_sep_fs,
          model_3:model_trainer_3,
          model_sep_3:model_trainer_sep_3,
          model_sep_fs_3:model_trainer_sep_fs_3,
          model_sep_32:model_trainer_sep_32,
          model_sep_4:model_trainer_sep_4,
          model_sep_fs_4:model_trainer_sep_fs_4}#,
          model_4:model_trainer_4}

In [ ]:
mod = model
trainer = models[mod]
plt.plot(trainer.hist_train.acc_list, label = 'g - train')
plt.plot(trainer.hist_val.acc_list, label = 'g - train')

In [ ]:
mod = model_sep_fs
trainer = models[mod]
plt.plot(trainer.hist_val.acc_c_g,color = 'tab:orange', label = 'g - val')
plt.axhline(trainer.hist_val.acc_c_mcp[-1],color = 'tab:blue', label = 'acc_0 - val')
plt.axhline(trainer.hist_val.acc_list[-1],color = 'tab:red', label = 'acc_0 - val')
#plt.plot(trainer.hist_train.acc_c_mcp, label = 'mcp - train')
#plt.plot(trainer.hist_train.acc_c_g, label = 'g - train')
#plt.axhline(trainer.hist_val.acc_list[-1],color = 'r', label = 'acc_0 - val')
plt.grid()
plt.legend()
plt.show()

plt.plot(trainer.hist_val.g_list)
plt.grid()
plt.show()

plt.plot(trainer.hist_val.loss_list,label = 'Validation')
plt.plot(trainer.hist_train.loss_list,label = 'Training')
plt.grid()
plt.legend()
plt.show()

In [ ]:
mod = model_sep_fs
trainer = models[mod]
#plt.plot(trainer.hist_train.acc_c_mcp, label = 'mcp - train')
plt.plot(trainer.hist_train.acc_c_g,color = 'tab:orange', label = 'g - train')
plt.axhline(trainer.hist_train.acc_c_mcp[-1],color = 'tab:blue', label = 'mcp - train')
plt.axhline(trainer.hist_train.acc_list[-1],color = 'tab:red', label = 'acc_0 - val')

plt.grid()
plt.legend()
plt.show()

plt.plot(trainer.hist_train.g_list)
plt.grid()
plt.show()

plt.plot(trainer.hist_val.loss_list,label = 'Validation')
plt.plot(trainer.hist_train.loss_list,label = 'Training')
plt.grid()
plt.legend()
plt.show()

In [ ]:
mod = model_sep_fs
label, output, g = accumulate_results(mod,train_dataloader)
output = torch.exp(output)
acc = correct_total(output,label)/label.size(0)
g_list = []
mcp_list = []
ideal = []
for c in np.arange(0,1,0.05):
    g_list.append(unc_comp.acc_coverage(output,label,1-g,c))
    mcp = unc.MCP_unc(output)
    mcp_list.append(unc_comp.acc_coverage(output,label,mcp,c))
    ideal.append(min(1,acc/(1-c)))


In [ ]:

plt.plot(np.arange(0,1,0.05),mcp_list,label = 'mcp')
plt.plot(np.arange(0,1,0.05),g_list, label = 'g')
plt.plot(np.arange(0,1,0.05),ideal,label = 'ideal')
plt.plot(np.arange(0,1,0.05),acc*np.ones(len(g_list)))
plt.grid()
plt.legend()
plt.show()